In [1]:
import sys
sys.path.append('..')

from src.data.make_dataset import *
from tqdm import tqdm

In [2]:
embeddings_index = parse_glove_data(r"..\data\raw\glove.6B.300d.txt")

In [3]:
words_array = get_words_array(embeddings_index)
print("len(words_array) (raw):\t\t\t" + '{:,}'.format(len(words_array)))
words_array = filter_alphabetic(words_array)
print("len(words_array) (filter_alphabetic):\t" + '{:,}'.format(len(words_array)))
words_array = filter_20k(words_array, r'..\data\external\20k.txt')
print("len(words_array) (filter_20k):\t\t" + '{:,}'.format(len(words_array)))

len(words_array) (raw):			400,000
len(words_array) (filter_alphabetic):	317,756
len(words_array) (filter_20k):		19,737


In [4]:
np.random.choice(words_array, size=20, replace=False)

array(['spies', 'todos', 'psc', 'services', 'jd', 'acorn', 'lastly',
       'greeting', 'firefly', 'listing', 'uncomfortable', 'nevada',
       'prediction', 'sphere', 'plunge', 'furnished', 'elder', 'spyware',
       'ist', 'unpredictable'], dtype='<U39')

In [45]:
glove_df = embeddings_to_dataframe(embeddings_index, words_array)

In [6]:
starting_word = np.random.choice(words_array)
factor_words = np.random.choice(words_array, size=2, replace=False)
operations = np.random.choice([-1, 1], size=2)

factor_matrix = np.array([embeddings_index[word] for word in factor_words])
factors_x_operations = np.dot(operations, factor_matrix)

result_vec = embeddings_index[starting_word] + factors_x_operations
# final_words = list(glove_df.index[np.dot(glove_df, result_vec).argsort()[::-1][:5]])
final_words = glove_df.index[np.dot(glove_df, result_vec).argsort()[::-1][0]]

op_text = [" + " if o == 1 else " - " for o in list(operations)]
print(starting_word + op_text[0] + factor_words[0]
                    + op_text[1] + factor_words[1]
                    # + op_text[2] + factor_words[2]
                    + " = " + str(final_words))

nist - co - contracted = nist


In [71]:
def random_similar_word(word, glove_df, similarity_threshold=200):
    sorted_similarities = np.dot(glove_df, embeddings_index[word]).argsort()[::-1]
    filtered_glove = glove_df.index[sorted_similarities[1:similarity_threshold+1]]
    similar_word = np.random.choice(filtered_glove)
    return similar_word
        
random_similar_word('king', glove_df)

'egypt'

In [74]:
def simulate_game(words_array, glove_df, num_operations):
    starting_word = np.random.choice(words_array)
    # factor_words = np.random.choice(words_array, size=num_operations, replace=False)
    factor_words = np.array([random_similar_word(starting_word, glove_df, 100) for _ in range(num_operations)])
    operations = np.random.choice([-1, 1], size=num_operations)
    # operations = np.array([-1, 1])

    factor_matrix = np.array([embeddings_index[word] for word in factor_words])
    factors_x_operations = np.dot(operations, factor_matrix)

    result_vec = embeddings_index[starting_word] + factors_x_operations

    num_words = num_operations + 2
    final_words = glove_df.index[np.dot(glove_df, result_vec).argsort()[::-1][:num_words]]
    final_word = final_words[0]
    i = 0
    while(final_word == starting_word or final_word in factor_words):
        i += 1
        final_word = final_words[i]

    similarity = np.dot(embeddings_index[final_word], result_vec)
    equation_string = starting_word
    for i in range(num_operations):
        if operations[i] == 1:
            equation_string += ' + '
        else:
            equation_string += ' - '
        equation_string += factor_words[i]
    equation_string += ' = ' + final_word

    return similarity, equation_string

simulate_game(words_array, glove_df, 2)

(29.300819491447566, 'parts - divided + some = equipment')

In [75]:
sim_results = []
num_operations = 2
num_results = 100
threshold = 30

with tqdm(total=num_results, desc="Equations found") as pbar:
    while len(sim_results) < num_results:
        sim = simulate_game(words_array, glove_df, num_operations)
        if sim[0] > threshold:
            sim_results.append(sim)
            pbar.update(1)
results_df = pd.DataFrame(sim_results, columns=["similarity", "equation"])

sorted_results = results_df.sort_values(by="similarity", ascending=False)
sorted_results.to_csv(r'..\data\processed\results.csv')

Equations found: 100%|██████████| 100/100 [00:07<00:00, 12.71it/s]


In [ ]:
print(np.dot(embeddings_index['king'] - embeddings_index['man'] + embeddings_index['woman'], embeddings_index['queen']))
print(np.dot(embeddings_index['king'], embeddings_index['queen']))